# Ecobici Datos 

In [18]:
#Crear la sesión en Spark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EcoBiciTarea").getOrCreate()


In [19]:
#Crear el df
csv_directory = "/home/atenea/Macrodatos/Tarea Ecobici"
df = spark.read.csv(f"{csv_directory}/*.csv", header=True, inferSchema=True)


In [20]:
df.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)



In [21]:
df.show(10)

+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|         Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|         Hora_Arribo|
+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+
|             M|          28|2401|                  118|  2012-12-01|2024-10-20 00:00:...|                  118|  2012-12-01| 2024-10-20 00:15:00|
|             M|          26|1540|                   73|  2012-12-01|2024-10-20 00:00:...|                   63|  2012-12-01|2024-10-20 00:06:...|
|             F|          30|2197|                   64|  2012-12-01|2024-10-20 00:00:...|                  167|  2012-12-01| 2024-10-20 00:16:21|
|             M|          27|1421|                   63|  2012-12-01|2024-10-20 00:00:...|                   53|  2012

## Añadir columnas para Fecha y Hora de retiro

In [22]:
from pyspark.sql import functions as F
# Combinar Fecha y Hora de Retiro
df_Ret = df.withColumn("fecha_hora_retiro", F.concat_ws(" ", "Fecha_Retiro", F.date_format("Hora_Retiro", "HH:mm:ss"))) \
            .withColumn("fecha_hora_retiro", F.to_timestamp("fecha_hora_retiro"))

# Extraer año, mes, día, minuto y segundo
df_Ret = df_Ret.withColumn("anio_retiro", F.year("fecha_hora_retiro")) \
                .withColumn("mes_retiro", F.month("fecha_hora_retiro")) \
                .withColumn("dia_retiro", F.dayofmonth("fecha_hora_retiro")) \
                .withColumn("minuto_retiro", F.minute("fecha_hora_retiro")) \
                .withColumn("segundo_retiro", F.second("fecha_hora_retiro"))

df_Ret.show(10)

+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|         Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|         Hora_Arribo|  fecha_hora_retiro|anio_retiro|mes_retiro|dia_retiro|minuto_retiro|segundo_retiro|
+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|             M|          28|2401|                  118|  2012-12-01|2024-10-20 00:00:...|                  118|  2012-12-01| 2024-10-20 00:15:00|2012-12-01 00:00:14|       2012|        12|         1|            0|            14|
|             M|          26|1540|                   73|  2012-12-01|2024-10-20 

In [23]:
df_Ret.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)
 |-- fecha_hora_retiro: timestamp (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- mes_retiro: integer (nullable = true)
 |-- dia_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)



## Añadir columnas para Fecha y Hora de arribo

In [24]:
from pyspark.sql import functions as F

# Crear la columna fecha_hora_arribo
df_Arr = df.withColumn("fecha_hora_arribo", F.concat_ws(" ", "Fecha_Arribo", F.date_format("Hora_Arribo", "HH:mm:ss"))) \
           .withColumn("fecha_hora_arribo", F.to_timestamp("fecha_hora_arribo"))

# Extraer año, mes, día, minuto y segundo para fecha_hora_arribo
df_Arr = df_Arr.withColumn("anio_arribo", F.year("fecha_hora_arribo")) \
               .withColumn("mes_arribo", F.month("fecha_hora_arribo")) \
               .withColumn("dia_arribo", F.dayofmonth("fecha_hora_arribo")) \
               .withColumn("minuto_arribo", F.minute("fecha_hora_arribo")) \
               .withColumn("segundo_arribo", F.second("fecha_hora_arribo"))

# Mostrar el DataFrame resultante
df_Arr.show(10)


+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|         Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|         Hora_Arribo|  fecha_hora_arribo|anio_arribo|mes_arribo|dia_arribo|minuto_arribo|segundo_arribo|
+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|             M|          28|2401|                  118|  2012-12-01|2024-10-20 00:00:...|                  118|  2012-12-01| 2024-10-20 00:15:00|2012-12-01 00:15:00|       2012|        12|         1|           15|             0|
|             M|          26|1540|                   73|  2012-12-01|2024-10-20 

In [25]:
df_Arr.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)
 |-- fecha_hora_arribo: timestamp (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- mes_arribo: integer (nullable = true)
 |-- dia_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)



### Muestra de sólo las columnas que se acaban de añadir

In [26]:
# Seleccionar las columnas de arribo y retiro
df_combinado = df_Arr.join(
    df_Ret,
    on="Bici",  
    how="inner"  
).select(
    "fecha_hora_arribo",
    "anio_arribo",
    "mes_arribo",
    "dia_arribo",
    "minuto_arribo",
    "segundo_arribo",
    "fecha_hora_retiro",
    "anio_retiro",
    "mes_retiro",
    "dia_retiro",
    "minuto_retiro",
    "segundo_retiro"
)

# Mostrar el DataFrame combinado
df_combinado.show()


+-------------------+-----------+----------+----------+-------------+--------------+-------------------+-----------+----------+----------+-------------+--------------+
|  fecha_hora_arribo|anio_arribo|mes_arribo|dia_arribo|minuto_arribo|segundo_arribo|  fecha_hora_retiro|anio_retiro|mes_retiro|dia_retiro|minuto_retiro|segundo_retiro|
+-------------------+-----------+----------+----------+-------------+--------------+-------------------+-----------+----------+----------+-------------+--------------+
|2012-12-01 13:58:40|       2012|        12|         1|           58|            40|2012-12-01 13:43:43|       2012|        12|         1|           43|            43|
|2012-12-01 13:58:40|       2012|        12|         1|           58|            40|2012-12-02 07:33:39|       2012|        12|         2|           33|            39|
|2012-12-01 13:58:40|       2012|        12|         1|           58|            40|2012-12-03 08:50:18|       2012|        12|         3|           50|        

## Crear tablas temporales

In [27]:
df_combinado = df_combinado.select(
    "fecha_hora_arribo",
    "anio_arribo",
    "mes_arribo",
    "dia_arribo",
    "minuto_arribo",
    "segundo_arribo",
    "fecha_hora_retiro",
    "anio_retiro",
    "mes_retiro",
    "dia_retiro",
    "minuto_retiro",
    "segundo_retiro"
)
df_temp = df.select(
    "Genero_Usuario",
    "Edad_Usuario",
    "Bici",
    "Ciclo_Estacion_Retiro"
)
df_temp = df_temp.join(df_combinado)

df_temp.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- fecha_hora_arribo: timestamp (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- mes_arribo: integer (nullable = true)
 |-- dia_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)
 |-- fecha_hora_retiro: timestamp (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- mes_retiro: integer (nullable = true)
 |-- dia_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)



In [28]:
df_temp.createOrReplaceTempView("Ecobicitemporal")

## Gráficas de viajes

### Grafica de viajes durante el mes de enero

In [29]:
import pyspark
print(pyspark.__version__)

3.5.3


In [30]:
df_temp.show(5, truncate=False)

+--------------+------------+----+---------------------+-------------------+-----------+----------+----------+-------------+--------------+-------------------+-----------+----------+----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|fecha_hora_arribo  |anio_arribo|mes_arribo|dia_arribo|minuto_arribo|segundo_arribo|fecha_hora_retiro  |anio_retiro|mes_retiro|dia_retiro|minuto_retiro|segundo_retiro|
+--------------+------------+----+---------------------+-------------------+-----------+----------+----------+-------------+--------------+-------------------+-----------+----------+----------+-------------+--------------+
|M             |28          |2401|118                  |2012-12-01 13:58:40|2012       |12        |1         |58           |40            |2012-12-01 13:43:43|2012       |12        |1         |43           |43            |
|M             |28          |2401|118                  |2012-12-01 13:58:40|2012       |12        |1        

In [31]:
# Filtrar los datos solo para el mes de enero
# Acceso correcto a la columna
df_enero = df_temp.filter(df_temp['mes_retiro'] == 1)

# Contar los viajes por día de retiro
viajes_enero = df_enero.groupBy('fecha_hora_retiro').count().orderBy('fecha_hora_retiro')

df_enero.printSchema()



root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- fecha_hora_arribo: timestamp (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- mes_arribo: integer (nullable = true)
 |-- dia_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)
 |-- fecha_hora_retiro: timestamp (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- mes_retiro: integer (nullable = true)
 |-- dia_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)



In [32]:
# Recoger los datos como una lista de filas
viajes_enero_data = viajes_enero.collect()
fechas = [row['fecha_retiro'] for row in viajes_enero_data]
conteos = [row['count'] for row in viajes_enero_data]

# Crear la gráfica
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(fechas, conteos, color='blue')
plt.xlabel('Fecha de Retiro')
plt.ylabel('Número de Viajes')
plt.title('Número de Viajes por Fecha de Retiro en Enero')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

24/10/20 12:51:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 12:51:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 12:51:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 12:51:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 12:51:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 12:51:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 12:51:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


[1064.765s][warning][gc,alloc] Executor task launch worker for task 9.0 in stage 35.0 (TID 190): Retried waiting for GCLocker too often allocating 4194306 words
[1064.794s][warning][gc,alloc] Executor task launch worker for task 7.0 in stage 35.0 (TID 188): Retried waiting for GCLocker too often allocating 4194306 words


24/10/20 12:51:18 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:18 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:18 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:18 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:18 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:19 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:19 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:19 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:19 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
24/10/20 12:51:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/20 1

[1066.918s][warning][gc,alloc] Executor task launch worker for task 0.0 in stage 35.0 (TID 181): Retried waiting for GCLocker too often allocating 131074 words


24/10/20 12:51:21 WARN RowBasedKeyValueBatch: Failed to allocate page (2097136 bytes).
24/10/20 12:51:21 ERROR Executor: Exception in task 1.0 in stage 35.0 (TID 182)
java.lang.OutOfMemoryError: Java heap space
24/10/20 12:51:21 ERROR Executor: Exception in task 0.0 in stage 35.0 (TID 181)
java.lang.OutOfMemoryError: Java heap space
24/10/20 12:51:21 ERROR Executor: Exception in task 10.0 in stage 35.0 (TID 191)
java.lang.OutOfMemoryError: Java heap space
24/10/20 12:51:21 ERROR Executor: Exception in task 2.0 in stage 35.0 (TID 183)
java.lang.OutOfMemoryError: Java heap space
24/10/20 12:51:21 ERROR Executor: Exception in task 9.0 in stage 35.0 (TID 190)
java.lang.OutOfMemoryError: Java heap space
24/10/20 12:51:21 WARN TaskMemoryManager: Failed to allocate a page (8388608 bytes), try again.
24/10/20 12:51:21 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 0.0 in stage 35.0 (TID 181),5,main]
java.lang.OutOfMemoryError: Java

ConnectionRefusedError: [Errno 111] Connection refused